# Mapserver / Mapfile

In [1]:
# local
from AlertaDengue.dados import dbdata

import os
import sqlalchemy as sqla
# local
import AlertaDengue as alert_dengue

## Setting variables

In [2]:
shp_path = '%s/static/shapefile' % alert_dengue.__path__[0]
ms_error_path = '/home/xmn/alerta_dengue_ms.log'
ms_cgi_path = 'http://localhost:81/cgi-bin/mapserv?map=%s&'
ms_mapfile_name = '%s.map'
ms_mapfile_dir = '%s/mapfiles/%%s' % (
    os.path.dirname(alert_dengue.__path__[0])
)

In [3]:
diseases = ('dengue', 'chikungunya')

In [4]:
alert_colors = [
    '#00FF00',
    '#FFFF00',
    '#FF9900',
    '#FF0000',
]

## DB connection

In [5]:
def get_passwd_from_ini(filepath: str):
    with open(filepath) as f:
        text = f.read()
        i = text.index('PSQL_PASSWORD')
        f = text[i:].index('\n')
        passwd = text[i:i+f].split('=')
        return passwd[1].strip()
    

In [6]:
db = 'dengue'
user = 'dengueadmin'
host = 'localhost'
passwd = get_passwd_from_ini('../AlertaDengue/AlertaDengue/settings.ini')
dsn = "postgresql://{}:{}@{}/{}".format(
    user, passwd, host, db
)

engine = sqla.create_engine(dsn)

## Mapfile

### Templates

In [7]:
# mapfile templates

mapfile_template = '''
MAP
    # The geographic extent (the rectangular area covered by the map) is 
    # defined by the keyword EXTENT. The rectangular area is specified by
    # the coordinates of the opposite corners (the lower left and the upper 
    # right). These are coordinates of the southwest and the northeast corners 
    # EX:
    # EXTENT -125.00 20.00 -65.00 50.00
    # The geographic extent stretches from 125° west, 20° north to 65° west, 
    # 50° north.
    
    CONFIG 'ON_MISSING_DATA' 'IGNORE'
    CONFIG 'PROJ_LIB' './conf/'
    CONFIG      "MS_ERRORFILE" "%(ms_error_path)s"
    CONFIG      "CPL_DEBUG" "ON"
    CONFIG      "PROJ_DEBUG" "ON"
    DEBUG       5
    
    IMAGETYPE   png
    IMAGECOLOR  0 0 0
    MAXSIZE     4000
    SIZE        800 800
    UNITS       meters
    EXTENT -5975305.2003561 -3046678.8822135 -4208595.4045127 -2061227.68955711
    
    OUTPUTFORMAT
      NAME      "png"
      DRIVER    AGG/PNG
      MIMETYPE  "image/png"
      IMAGEMODE RGBA
      EXTENSION "png"
      FORMATOPTION "GAMMA=0.75"
    END
    
    # OUTPUTFORMAT
    #   NAME "GTiff"
    #   DRIVER GDAL/GTiff
    #   MIMETYPE "image/tiff"
    #   IMAGEMODE RGBA
    #   EXTENSION "tif"
    # END
    
    # OUTPUTFORMAT
    #   NAME "kml"
    #   DRIVER KML
    #   MIMETYPE "application/vnd.google-earth.kml.xml"
    #   IMAGEMODE RGBA
    #   EXTENSION "kml"
    # END

    PROJECTION
        "init=epsg:900913"
    END

    WEB
      METADATA
        "wms_title" "Alerta Dengue"
        "wms_onlineresource" "%(ms_cgi_path)s"
        "ows_enable_request" "*"
        "wms_srs" "EPSG:900913 EPSG:4326"
        "wms_feature_info_mime_type" "text/html"
        "wms_format" "image/png"
      END
      
      IMAGEPATH '/tmp/map/'
      IMAGEURL '/mapimg/'
    END
    
    SHAPEPATH '%(shp_path)s/'
 
%(include_layers)s
 
END
'''

mapfile_layer_template = '''
    LAYER
        NAME         "%(city_name)s"
        DATA         "%(geocode)s"
        STATUS       ON
        TYPE         POLYGON
        # OFFSITE      0 0 0
        
        PROJECTION
          "init=epsg:4326"
        END
        
        METADATA
          "wms_srs" "EPSG:4326"
          "wms_include_items" "all" 
        END
        
        # COMPOSITE
        #     OPACITY 70
        # END # COMPOSITE
 
        CLASS
            STYLE
                COLOR '%(rgb)s'
            END
        END
    END
'''

### Generating the mapfile

In [8]:
sql_template = '''
SELECT geocodigo, nome, uf 
FROM "Dengue_global"."Municipio" 
WHERE uf = '%s'
ORDER BY nome;
'''

In [9]:
state_initials = {
    'Paraná': 'PR', 
    'Rio de Janeiro': 'RJ', 
    'Ceará': 'CE', 
    'Minas Gerais': 'MG', 
    'Espírito Santo': 'ES'
}

In [10]:
# prepare mapfile
layers = {}

for disease in diseases:
    layers[disease] = {}
    
    # layers by state
    for state_name in state_initials.keys():
        with engine.connect() as conn:
            sql = sql_template % state_name
            result = conn.execute(sql).fetchall()

        cities_alert = dbdata.NotificationResume.get_cities_alert_by_state(
            state_name, disease
        )

        alerts = dict(
            cities_alert[['municipio_geocodigo', 'level_alert']].values
        )
        
        layers[disease][state_name] = []
        
        for geocode, city_name, state_country in result:
            alert_level = alerts[geocode] if geocode in alerts else -1
            # print(alert_level)
            alert_color = (
                alert_colors[alert_level] if 0 <= alert_level <= 3 else
                '#DFDFDF'  # gray
            )
            layers[disease][state_name] += [{
                'geocode': geocode,
                'city_name': city_name,
                'rgb': alert_color
            }]

In [11]:
# save mapfile

include_layers = {}
include_template = '    INCLUDE "layers/%s"  # %s\n'

# save individual layers
for disease, states_layer in layers.items():
    include_layers[disease] = {}
    
    for state_name, layers_conf in states_layer.items():
        include_layers[disease][state_name] = ''
        for layer_conf in layers_conf:
            layer_content = mapfile_layer_template % layer_conf
            layer_name = '%s-%s.map' % (disease, layer_conf['geocode'])
            layer_path = ms_mapfile_dir % ('/layers/%s' % layer_name)
            
            include_layer = include_template % (
                layer_name, layer_conf['city_name']
            )
            
            if not os.path.exists(
                os.path.join(shp_path, '%s.shp' % layer_conf['geocode'])
            ):
                include_layer = '#' + include_layer

            include_layers[disease][state_name] += include_layer

            with open(layer_path, 'w') as f:
                f.write(layer_content)

                
        # save for individual state
        mapfile_name = ms_mapfile_name % (
            '%s-%s' % (disease, state_initials[state_name])
        )
        mapfile_path = ms_mapfile_dir % mapfile_name

        ms_config = {
            'include_layers': include_layers[disease][state_name],
            'ms_error_path': ms_error_path,
            'ms_cgi_path': ms_cgi_path % mapfile_path,
            'shp_path': shp_path
        }

        mapfile_content = mapfile_template % ms_config

        print('Saving ', mapfile_path, 'file ...')
        with open(mapfile_path, 'w') as f:
            f.write(mapfile_content)
    
    # save mapfile with all cities
    mapfile_name = ms_mapfile_name % disease
    mapfile_path = ms_mapfile_dir % mapfile_name
    
    ms_config = {
        'include_layers': ''.join(include_layers[disease].values()),
        'ms_error_path': ms_error_path,
        'ms_cgi_path': ms_cgi_path % mapfile_path,
        'shp_path': shp_path
    }
    
    mapfile_content = mapfile_template % ms_config
    
    print('Saving ', mapfile_path, 'file ...')
    with open(mapfile_path, 'w') as f:
        f.write(mapfile_content)

print('[II] DONE!')

Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/dengue-PR.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/dengue-RJ.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/dengue-CE.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/dengue-MG.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/dengue-ES.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/dengue.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/chikungunya-PR.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/chikungunya-RJ.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/chikungunya-CE.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/chikungunya-MG.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/chikungunya-ES.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/chikungunya.map file ...
[II] DONE!


## Visualizing the map

In [14]:
%%html

<div id="alerta_dengue" style="height: 400px; width: 800px;"></div>

<link rel="stylesheet" href="lib/leaflet/leaflet.css" />
<script type="text/javascript" src="lib/leaflet/leaflet.js"></script>

<script>

var mapfile_url = (
    'http://localhost:81/cgi-bin/mapserv?' + 
    'map=/var/www/mapserver/mapfiles/dengue.map&' + 
    'mode=map&layers=all'
);

var map = L.map('alerta_dengue').setView([-14.01,-53.96], 4); 


var osmUrl='http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png';
var osmAttrib = 'Map data © <a href="http://openstreetmap.org">OpenStreetMap</a> contributors';

osm = L.tileLayer(osmUrl, {
    minZoom: 4, 
    maxZoom: 10, 
    attribution: osmAttrib
});
osm.addTo(map);

ms = L.tileLayer.wms(mapfile_url, {
    layers: 'Roads',
    format: 'image/png',
    transparent: true,
    attribution: "Info Dengue",
    maxZoom: 10,
    minZoom: 4,
});
ms.addTo(map);

var baseMap = {
    "OSM": osm
};

var overlayMaps = {
    "Rio Alerta": ms
};

L.control.layers(baseMap, overlayMaps).addTo(map);


</script>